# 第7章: データベース
artist.json.gzは，オープンな音楽データベースMusicBrainzの中で，アーティストに関するものをJSON形式に変換し，gzip形式で圧縮したファイルである．このファイルには，1アーティストに関する情報が1行にJSON形式で格納されている．JSON形式の概要は以下の通りである．

|フィールド|型|内容|例|
|--|--|--|--|
|id|ユニーク識別子|整数|20660|
|gid|グローバル識別子|文字列|"ecf9f3a3-35e9-4c58-acaa-e707fba45060"|
|name|アーティスト名|文字列|"Oasis"|
|sort_name|アーティスト名（辞書順整列用）|文字列|"Oasis"|
|area|活動場所|文字列|"United Kingdom"|
|aliases|別名|辞書オブジェクトのリスト||
|aliases[].name|別名|文字列|"オアシス"|
|aliases[].sort_name|別名（整列用）|文字列|"オアシス"|
|begin|活動開始日|辞書||
|begin.year|活動開始年|整数|1991|
|begin.month|活動開始月|整数||
|begin.date|活動開始日|整数||
|end|活動終了日|辞書||
|end.year|活動終了年|整数|2009|
|end.month|活動終了月|整数|8|
|end.date|活動終了日|整数|28|
|tags|タグ|辞書オブジェクトのリスト||
|tags[].count|タグ付けされた回数|整数|1|
|tags[].value|タグ内容|文字列|"rock"|
|rating|レーティング|辞書オブジェクト||
|rating.count|レーティングの投票数|整数|13|
|rating.value|レーティングの値（平均値）|整数|86|

artist.json.gzのデータをKey-Value-Store (KVS) およびドキュメント志向型データベースに格納・検索することを考える．KVSとしては，LevelDB，Redis，KyotoCabinet等を用いよ．ドキュメント志向型データベースとして，MongoDBを採用したが，CouchDBやRethinkDB等を用いてもよい．

## 60. KVSの構築
Key-Value-Store (KVS) を用い，アーティスト名（name）から活動場所（area）を検索するためのデータベースを構築せよ．

In [1]:
# Redisのインストール
# apt-get install redis-server

# redis-pyのインストール
# pip install redis

In [2]:
!head -n 10 ../data/artist.json

{"name": "WIK▲N", "tags": [{"count": 1, "value": "sillyname"}], "sort_name": "WIK▲N", "ended": true, "gid": "8972b1c1-6482-4750-b51f-596d2edea8b1", "id": 805192}
{"name": "Gustav Ruppke", "sort_name": "Gustav Ruppke", "ended": true, "gid": "b4f76788-7e6f-41b7-ac7b-dfb67f66282e", "type": "Person", "id": 578352}
{"name": "Pete Moutso", "sort_name": "Moutso, Pete", "ended": true, "gid": "49add228-eac5-4de8-836c-d75cde7369c3", "type": "Person", "id": 371203}
{"ended": true, "gid": "c112a400-af49-4665-8bba-741531d962a1", "sort_name": "Zachary", "id": 273232, "name": "Zachary"}
{"name": "The High Level Ranters", "sort_name": "High Level Ranters, The", "ended": true, "gid": "c42eed94-e233-44e2-82b8-3ed6dd9bf318", "type": "Group", "id": 153193}
{"begin": {"year": 1956}, "end": {"year": 1993}, "name": "The Silhouettes", "area": "United States", "sort_name": "Silhouettes, The", "ended": true, "gid": "ca3f3ee1-c4a7-4bac-a16a-0b888a396c6b", "type": "Group", "id": 101060, "aliases": [{"name": "Silh

In [3]:
!wc -l ../data/artist.json

921337 ../data/artist.json


In [4]:
import redis
import json

def generate_artist_info():
    with open('../data/artist.json') as f:
        for line in f:
            yield json.loads(line)

def knock_60():
    pool = redis.ConnectionPool(host='localhost', port=6379, db=0)
    r = redis.StrictRedis(connection_pool=pool)
    r.flushdb()
    for artist_info in generate_artist_info():
        name = artist_info['name']
        area = artist_info.get('area', '')
        # key: name, value: areaのSet
        r.sadd(name, area)
    print(str(r.dbsize()) + '件 登録' )

knock_60()

864789件 登録


## 61. KVSの検索
60で構築したデータベースを用い，特定の（指定された）アーティストの活動場所を取得せよ．

In [5]:
import redis

def get_areas(artist_name):
    pool = redis.ConnectionPool(host='localhost', port=6379, db=0)
    r = redis.StrictRedis(connection_pool=pool)
    return [area.decode('utf-8') for area in r.smembers(artist_name)]

print('Mogwai -> '+ ' / '.join(get_areas('Mogwai')))
print('Latvijas radio sieviešu vokālais ansamblis -> '+ ' / '.join(get_areas('Latvijas radio sieviešu vokālais ansamblis')))
print('임슬옹 -> '+ ' / '.join(get_areas('임슬옹')))
print('Door -> '+ ' / '.join(get_areas('Door')))

Mogwai -> Germany / United Kingdom
Latvijas radio sieviešu vokālais ansamblis -> Latvia
임슬옹 -> South Korea
Door -> 


## 62. KVS内の反復処理
60で構築したデータベースを用い，活動場所が「Japan」となっているアーティスト数を求めよ．

In [6]:
import redis

def knock_62():
    pool = redis.ConnectionPool(host='localhost', port=6379, db=0)
    r = redis.StrictRedis(connection_pool=pool)
    return [key for key in r.scan_iter(match='*', count=10000) if r.sismember(key, 'Japan')]

print(len(knock_62()))

22554


## 63. オブジェクトを値に格納したKVS
KVSを用い，アーティスト名（name）からタグと被タグ数（タグ付けされた回数）のリストを検索するためのデータベースを構築せよ．さらに，ここで構築したデータベースを用い，アーティスト名からタグと被タグ数を検索せよ．

In [7]:
import redis
import pickle
from collections import defaultdict
import pprint

def build_db_knock_63():
    pool = redis.ConnectionPool(host='localhost', port=6379, db=1)
    r = redis.StrictRedis(connection_pool=pool)
    r.flushdb()
    for artist_info in generate_artist_info():
        key = 'id:' + str(artist_info['id']) + ':name:' + artist_info['name']
        r.set(key, pickle.dumps(artist_info))

def get_tags(name):
    pool = redis.ConnectionPool(host='localhost', port=6379, db=1)
    r = redis.StrictRedis(connection_pool=pool)
    search_key_pattern = 'id:*:name:' + name
    all_tags = defaultdict(int)
    for hit_key in r.keys(search_key_pattern):
        artist_info = pickle.loads(r.get(hit_key))
        tags = artist_info.get('tags', [])
        for tag in tags:
            all_tags[tag['value']] += tag['count']
    return dict(all_tags)

pprint.pprint(get_tags('Mogwai'))

{'britannique': 1,
 'british': 2,
 'classic pop and rock': 1,
 'instrumental': 2,
 'instrumental rock': 1,
 'post-rock': 8,
 'rock': 1,
 'scottish': 3,
 'uk': 2}


## 64. MongoDBの構築
アーティスト情報（artist.json.gz）をデータベースに登録せよ．さらに，次のフィールドでインデックスを作成せよ: name, aliases.name, tags.value, rating.value

In [8]:
# 下記を参考に MongoDB をインストール
# https://docs.mongodb.com/manual/tutorial/install-mongodb-on-ubuntu/

# PyMongo のインストール
# pip install pymongo 

In [11]:
import pymongo
from itertools import islice

def build_db_knock_64():
    client = pymongo.MongoClient('mongodb://localhost:27017/')
    db_name = '100-knock'
    client.drop_database(db_name)
    db = client[db_name]
    col = db['artists']
    col.create_index([('name', pymongo.ASCENDING)])  
    col.create_index([('aliases.name', pymongo.ASCENDING)])  
    col.create_index([('tags.value', pymongo.ASCENDING)])
    col.create_index([('rating.value', pymongo.ASCENDING)])
    index = 0
    while True:
        chunk = tuple(islice(generate_artist_info(), index, index + 300000))
        if not chunk:
            break
        index += len(chunk)
        col.insert_many(chunk)
    print(str(col.count_documents({})) + '件 登録')
                
build_db_knock_64()

921337件 登録


## 65. MongoDBの検索
MongoDBのインタラクティブシェルを用いて，"Queen"というアーティストに関する情報を取得せよ．さらに，これと同様の処理を行うプログラムを実装せよ．

インタラクティブシェル
```
> mongo
> use 100-knock
> db.artists.find( {'name':'Queen'} )

{ "_id" : ObjectId("5bc494d938152d036d43a229"), "name" : "Queen", "area" : "Japan", "gender" : "Female", "tags" : [ { "count" : 1, "value" : "kamen rider w" }, { "count" : 1, "value" : "related-akb48" } ], "sort_name" : "Queen", "ended" : true, "gid" : "420ca290-76c5-41af-999e-564d7c71f1a7", "type" : "Character", "id" : 701492, "aliases" : [ { "name" : "Queen", "sort_name" : "Queen" } ] }

{ "_id" : ObjectId("5bc494da38152d036d4468d5"), "rating" : { "count" : 24, "value" : 92 }, "begin" : { "date" : 27, "month" : 6, "year" : 1970 }, "name" : "Queen", "area" : "United Kingdom", "tags" : [ { "count" : 2, "value" : "hard rock" }, { "count" : 1, "value" : "70s" }, { "count" : 1, "value" : "queen family" }, { "count" : 1, "value" : "90s" }, { "count" : 1, "value" : "80s" }, { "count" : 1, "value" : "glam rock" }, { "count" : 4, "value" : "british" }, { "count" : 1, "value" : "english" }, { "count" : 2, "value" : "uk" }, { "count" : 1, "value" : "pop/rock" }, { "count" : 1, "value" : "pop-rock" }, { "count" : 1, "value" : "britannique" }, { "count" : 1, "value" : "classic pop and rock" }, { "count" : 1, "value" : "queen" }, { "count" : 1, "value" : "united kingdom" }, { "count" : 1, "value" : "langham 1 studio bbc" }, { "count" : 1, "value" : "kind of magic" }, { "count" : 1, "value" : "band" }, { "count" : 6, "value" : "rock" }, { "count" : 1, "value" : "platinum" } ], "sort_name" : "Queen", "ended" : true, "gid" : "0383dadf-2a4e-4d10-a46a-e9e041da8eb3", "type" : "Group", "id" : 192, "aliases" : [ { "name" : "女王", "sort_name" : "女王" } ] }

{ "_id" : ObjectId("5bc494db38152d036d46232d"), "ended" : true, "gid" : "5eecaf18-02ec-47af-a4f2-7831db373419", "sort_name" : "Queen", "id" : 992994, "name" : "Queen" }
```

In [12]:
from pymongo import MongoClient
import pprint

def knock_65(name):
    client = pymongo.MongoClient('mongodb://localhost:27017/')
    db_name = '100-knock'
    db = client[db_name]
    col = db['artists']
    for artist_info in col.find({'name': name}):
        pprint.pprint(artist_info)

knock_65('Queen')

{'_id': ObjectId('5bce6c4c38152d0c2120129b'),
 'aliases': [{'name': 'Queen', 'sort_name': 'Queen'}],
 'area': 'Japan',
 'ended': True,
 'gender': 'Female',
 'gid': '420ca290-76c5-41af-999e-564d7c71f1a7',
 'id': 701492,
 'name': 'Queen',
 'sort_name': 'Queen',
 'tags': [{'count': 1, 'value': 'kamen rider w'},
          {'count': 1, 'value': 'related-akb48'}],
 'type': 'Character'}
{'_id': ObjectId('5bce6c4d38152d0c2120d947'),
 'aliases': [{'name': '女王', 'sort_name': '女王'}],
 'area': 'United Kingdom',
 'begin': {'date': 27, 'month': 6, 'year': 1970},
 'ended': True,
 'gid': '0383dadf-2a4e-4d10-a46a-e9e041da8eb3',
 'id': 192,
 'name': 'Queen',
 'rating': {'count': 24, 'value': 92},
 'sort_name': 'Queen',
 'tags': [{'count': 2, 'value': 'hard rock'},
          {'count': 1, 'value': '70s'},
          {'count': 1, 'value': 'queen family'},
          {'count': 1, 'value': '90s'},
          {'count': 1, 'value': '80s'},
          {'count': 1, 'value': 'glam rock'},
          {'count': 4, 'valu

## 66. 検索件数の取得
MongoDBのインタラクティブシェルを用いて，活動場所が「Japan」となっているアーティスト数を求めよ．

インタラクティブシェル
```
> mongo
> use 100-knock
> db.artists.distinct( 'name', {'area':'Japan'} ).length

22554
```

## 67. 複数のドキュメントの取得
特定の（指定した）別名を持つアーティストを検索せよ．

In [13]:
def knock_67(alias):
    client = pymongo.MongoClient('mongodb://localhost:27017/')
    db_name = '100-knock'
    db = client[db_name]
    col = db['artists']
    for artist_info in col.find({'aliases.name': alias}):
        pprint.pprint(artist_info)
        
knock_67('OASIS')

{'_id': ObjectId('5bce6c4d38152d0c2120fe5a'),
 'aliases': [{'name': 'OASIS', 'sort_name': 'OASIS'},
             {'name': 'オアシス', 'sort_name': 'オアシス'}],
 'area': 'United Kingdom',
 'begin': {'year': 1991},
 'end': {'date': 28, 'month': 8, 'year': 2009},
 'ended': True,
 'gid': '39ab1aed-75e0-4140-bd47-540276886b60',
 'id': 20660,
 'name': 'Oasis',
 'rating': {'count': 13, 'value': 86},
 'sort_name': 'Oasis',
 'tags': [{'count': 1, 'value': 'rock'},
          {'count': 3, 'value': 'britpop'},
          {'count': 4, 'value': 'british'},
          {'count': 1, 'value': 'uk'},
          {'count': 1, 'value': 'britannique'},
          {'count': 1, 'value': 'rock and indie'},
          {'count': 1, 'value': 'england'},
          {'count': 1, 'value': 'manchester'}],
 'type': 'Group'}


## 68. ソート
"dance"というタグを付与されたアーティストの中でレーティングの投票数が多いアーティスト・トップ10を求めよ．

In [14]:
def knock_68():
    client = pymongo.MongoClient('mongodb://localhost:27017/')
    db_name = '100-knock'
    db = client[db_name]
    col = db['artists']
    for artist_info in col.find({'tags.value': 'dance', 'rating.count': {'$exists': True}}).sort('rating.count', pymongo.DESCENDING).limit(10):
        print(artist_info['name'] + '\t' + str(artist_info['rating']['count']))
        
knock_68()

Madonna	26
Björk	23
The Prodigy	23
Rihanna	15
Britney Spears	13
Maroon 5	11
Adam Lambert	7
Fatboy Slim	7
Basement Jaxx	6
Cornershop	5


## 69. Webアプリケーションの作成
ユーザから入力された検索条件に合致するアーティストの情報を表示するWebアプリケーションを作成せよ．アーティスト名，アーティストの別名，タグ等で検索条件を指定し，アーティスト情報のリストをレーティングの高い順などで整列して表示せよ．